In [3]:
# These are standard modules
import time
import numpy as np
from scipy import linalg
import sympy as sym
import matplotlib.pyplot as plt
from IPython.display import display, Latex, Markdown
from sympy.physics import mechanics

# This is a custom interface to the pybullet simulator
import ae353_cmg

# For the notebook to "reload" the CMG script
import importlib
importlib.reload(ae353_cmg)

# Suppress the use of scientific notation when printing small numbers
np.set_printoptions(suppress=True)

pybullet build time: Jan 28 2022 20:18:15


Create an instance of the `Simulator` class, which is an interface to the [pybullet](http://pybullet.org) simulation engine.

There are several optional arguments:

* `display` (`True` or `False`) is whether or not to show the simulation window;
* `damping` (a non-negative floating-point number, default `0.`) is the coefficient of viscous friction at each joint;
* `load_mass` (a non-negative floating-point number, default `1.`) is the mass of the sphere attached to the platform;
* `roll` (an angle in radians, default `np.pi / 2`) is the roll angle of the ground.

**You must evaluate this cell only *once*.** If you want to start fresh with a new simulator, you must do `Kernel -> Restart` from the notebook menu first. If you evaluate this cell more than once without a call to `Kernel -> Restart` then you may get strange behavior that is hard to debug.

In [4]:
simulator = ae353_cmg.Simulator(
    display=True,
    damping=0.,
    load_mass=1.,
    roll=(np.pi / 2),
)

In [11]:
# Define parameter values

# one principal moment of inertia of the platform
J1 = 0.5 # kg*m^2

# two principal moments of inertia of the gimbal
J2 = 0.001 # kg*m^2

# principal moments of inertia of the rotor
J3 = 0.01 # kg*m^2

# mass of the boom
m = 1. # kg

# length of the boom
r = 2. # m

# acceleration due to gravity
g = 9.81 # m/s^2

# velocity of rotor
v_rotor = 1000.


# composite parameters
a1 = -J3 + 2*J3
a2 = 2.*J3
a3 = -2.*g*m*r
a4 = 2.*J1 + 2.*J2 + 2.*m*r**2.
a5 = 2.*J3
a6 = (J3 - J3)/(2*(J2 + J3))
a7 = -J3/(J2 + J3)
a8 = 1./(J2 + J3)

In [12]:
# Define a function to return a state-space model for a given choice of equilibrium point.
def get_model(q1e, v1e, q2e, v2e, taue):
    # Define variables
    q1, v1, q2, v2, tau = sym.symbols('q1, v1, q2, v2, tau')
    
    # Define right-hand side of ODEs
    f = sym.Matrix([
        v1, 
        ((a1 * sym.sin(2 * q2) * v1 * v2) + (a2 * sym.cos(q2) * v2 * v_rotor) + (a3) * sym.sin(q1)) / (a4 + (a5 * sym.cos(q2)**2)),
        v2,
        a6 * sym.sin(2 * q2) * v1**2 + a7 * sym.cos(q2) * v1 * v_rotor + a8 * tau
    ])
    
    # Convert floating-point to rational numbers
    f = sym.nsimplify(f, rational=True)
    
    # Verify equilibrium point
    f_num = sym.lambdify([q1, v1, q2, v2, tau], f)
    if not np.allclose(f_num(q1e, v1e, q2e, v2e, taue), 0.):
        raise Exception('equilibrium point is invalid')
    
    # Find A and B in symbolic form
    A_sym = f.jacobian([q1, v1, q2, v2])
    B_sym = f.jacobian([tau])
    
    # Create lambda functions to allow numerical evaluation of A and B
    A_num = sym.lambdify([q1, v1, q2, v2, tau], A_sym)
    B_num = sym.lambdify([q1, v1, q2, v2, tau], B_sym)
    
    # Find A and B in numeric form (making sure the result is floating-point)
    A = A_num(q1e, q2e, v1e, v2e, taue).astype(float)
    B = B_num(q1e, q2e, v1e, v2e, taue).astype(float)
    
    # Return A and B
    return A, B



# Define a controller that implements linear state feedback.
class Controller:
    def __init__(self, K, q1e, v1e, q2e, v2e, taue):
        self.K = K
        self.q1e = q1e
        self.v1e = v1e
        self.q2e = q2e
        self.v2e = v2e
        self.taue = taue
    
    def reset(self):
        pass
    
    def run(
            self,
            t,
            platform_angle,
            platform_velocity,
            gimbal_angle,
            gimbal_velocity,
            rotor_velocity,
        ):
        
        # Find state
        x = np.array([
            platform_angle - self.q1e,
            platform_velocity - self.v1e,
            gimbal_angle - self.q2e,
            gimbal_velocity - self.v2e,
        ])
        
        # Find input
        u = - self.K @ x
        
        # Find actuator command
        tau = u[0]
        
        gimbal_torque = -tau
        return gimbal_torque
    
    
    
# Define a function to get data from simulation for a given choice
# of equilibrium point, gain matrix, and initial condition.
def get_data(simulator, q1e, v1e, q2e, v2e, taue, K, q1i, v1i, q2i, v2i, max_time=5.0):
    # Create controller
    controller = Controller(K, q1e, v1e, q2e, v2e, taue)

    # Reset simulator with initial conditions [q1i, v1i, q2i, v2i]
    simulator.reset(
        platform_angle = q1i,
        platform_velocity = v1i,
        gimbal_angle = q2i,
        gimbal_velocity = v2i,
        rotor_velocity = v_rotor,
    )

    # Reset controller
    controller.reset()
    
    # Choose a camera view ( simulator.camera_sideview() or simulator.camera_topview() )
    simulator.camera_sideview()

    # Run simulator
    data = simulator.run(
        controller,           # <-- required (an instance of your Controller class)
        max_time=max_time,    # <-- optional (how long you want to run the simulation in seconds)
        data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
        video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
    )

    # Return data
    return data



# Define a function to find numeric solution to closed-loop (linear) system.
def get_x_numeric(data, q1e, v1e, q2e, v2e, A, B):
    # Define initial conditions
    x0 = np.array([
        data['platform_angle'][0] - q1e,
        data['platform_velocity'][0] - v1e,
        data['gimbal_angle'][0] - q2e,
        data['gimbal_velocity'][0] - v2e,
    ])

    # Create empty array to hold state at each time
    x = []

    # Compute state at each time
    for t in data['t']:
        x.append(linalg.expm((A - B @ K) * t) @ x0)
    x = np.array(x)
    
    # Return state at each time
    return x



# Define a function to find symbolic solution to closed-loop (linear) system.
def get_x_symbolic(A, B, K):
    # Model
    A = sym.nsimplify(sym.Matrix(A), rational=True)
    B = sym.nsimplify(sym.Matrix(B), rational=True)
    K = sym.nsimplify(sym.Matrix(K), rational=True)
    
    # Variables
    t, x_1i, x_2i, x_3i, x_4i = sym.symbols('t, x_1i, x_2i, x_3i, x_4i', real=True)
    
    # Initial condition
    x0 = sym.Matrix([x_1i, x_2i, x_3i, x_4i])
    
    # Solution
    x = sym.exp((A - B @ K) * t) @ x0
    return x



# Define a function to show symbolic solution to closed-loop (linear) system.
def show_x_symbolic(A, B, K):
    x = get_x_symbolic(A, B, K)
    print('\nSOLUTION TO CLOSED-LOOP SYSTEM')
    display(Markdown(f'$$x(t) = {sym.latex(x)}$$'))

    
    
# Define a function to show results.
def show_results(data, q1e, v1e, q2e, v2e, taue, A, B, K, q1i, v1i, q2i, v2i, show_pred=False):
    t = data['t']
    q1 = data['platform_angle']
    v1 = data['platform_velocity']
    q2 = data['gimbal_angle']
    v2 = data['gimbal_velocity']
    x1 = q1 - q1e
    x2 = v1 - v1e
    x3 = q2 - q2e
    x4 = v2 - v2e
    
    fig, ((ax_q1, ax_x1), (ax_v1, ax_x2), (ax_q2, ax_x3), (ax_v2, ax_x4)) = plt.subplots(4, 2, figsize=(15, 12), sharex=True)

    ax_q1.plot(t, q1, label='$q_1$', linewidth=4)
    ax_q1.plot(t, np.ones_like(t) * q1e, '--', label='$q_1$ (desired)', linewidth=4)
    
    ax_v1.plot(t, v1, label='$v_1$', linewidth=4)
    ax_v1.plot(t, np.ones_like(t) * v1e, '--', label='$v_1$ (desired)', linewidth=4)
    
    ax_q2.plot(t, q2, label='$q_2$', linewidth=4)
    ax_q2.plot(t, np.ones_like(t) * q2e, '--', label='$q_2$ (desired)', linewidth=4)
    
    ax_v2.plot(t, v2, label='$v_2$', linewidth=4)
    ax_v2.plot(t, np.ones_like(t) * v2e, '--', label='$v_2$ (desired)', linewidth=4)
    
    ax_x1.plot(t, x1, label='$x_1$', linewidth=4)
    ax_x1.plot(t, np.zeros_like(t), '--', label='$x_1$ (desired)', linewidth=4)
    
    ax_x2.plot(t, x2, label='$x_2$', linewidth=4)
    ax_x2.plot(t, np.zeros_like(t), '--', label='$x_2$ (desired)', linewidth=4)
    
    ax_x3.plot(t, x3, label='$x_3$', linewidth=4)
    ax_x3.plot(t, np.zeros_like(t), '--', label='$x_3$ (desired)', linewidth=4)
    
    ax_x4.plot(t, x4, label='$x_4$', linewidth=4)
    ax_x4.plot(t, np.zeros_like(t), '--', label='$x_4$ (desired)', linewidth=4)
    
    if show_pred:
        # Get x(t) and extract components
        x_num = get_x_numeric(data, q1e, v1e, q2e, v2e, A, B)
        x1_num = x_num[:, 0]
        x2_num = x_num[:, 1]
        x3_num = x_num[:, 2]
        x4_num = x_num[:, 3]
        
        # Compute q1(t), v1(t), q2(t) and v2(t) from x(t)
        q1_num = x1_num + q1e
        v1_num = x2_num + v1e
        q2_num = x3_num + q2e
        v2_num = x4_num + v2e
        
        # Add everything to plots
        ax_q1.plot(t, q1_num, ':', label='$q_1$ (linear)', linewidth=6, color='C3')
        ax_v1.plot(t, v1_num, ':', label='$v_1$ (linear)', linewidth=6, color='C3')
        ax_q2.plot(t, q2_num, ':', label='$q_2$ (linear)', linewidth=6, color='C3')
        ax_v2.plot(t, v2_num, ':', label='$v_2$ (linear)', linewidth=6, color='C3')
        ax_x1.plot(t, x1_num, ':', label='$x_1$ (linear)', linewidth=6, color='C3')
        ax_x2.plot(t, x2_num, ':', label='$x_2$ (linear)', linewidth=6, color='C3')
        ax_x3.plot(t, x3_num, ':', label='$x_3$ (linear)', linewidth=6, color='C3')
        ax_x4.plot(t, x4_num, ':', label='$x_4$ (linear)', linewidth=6, color='C3')
        
    ax_q1.grid()
    ax_q1.legend(fontsize=16)
    ax_q1.tick_params(labelsize=14)
    ax_q1.set_ylim(np.pi - 0.5, np.pi + 0.5)
    
    ax_v1.grid()
    ax_v1.legend(fontsize=16)
    ax_v1.tick_params(labelsize=14)
    ax_v1.set_ylim(-1, 1)
    
    ax_q2.grid()
    ax_q2.legend(fontsize=16)
    ax_q2.tick_params(labelsize=14)
    ax_q2.set_ylim(np.pi - 0.5, np.pi + 0.5)
    
    ax_v2.grid()
    ax_v2.legend(fontsize=16)
    ax_v2.tick_params(labelsize=14)
    ax_v2.set_ylim(-5, 5)

    ax_x1.grid()
    ax_x1.legend(fontsize=16)
    ax_x1.tick_params(labelsize=14)
    ax_x1.set_ylim(-0.5, 0.5)
        
    ax_x2.grid()
    ax_x2.legend(fontsize=16)
    ax_x2.tick_params(labelsize=14)
    ax_x2.set_ylim(-1, 1)
    
    ax_x3.grid()
    ax_x3.legend(fontsize=16)
    ax_x3.tick_params(labelsize=14)
    ax_x3.set_ylim(-5, 5)
    
    ax_x4.grid()
    ax_x4.legend(fontsize=16)
    ax_x4.tick_params(labelsize=14)
    ax_x4.set_ylim(-5, 5)
    
    ax_v2.set_xlabel('time (s)', fontsize=20)
    ax_v2.set_xlim([data['t'][0], data['t'][-1]])
    ax_x4.set_xlabel('time (s)', fontsize=20)
    ax_x4.set_xlim([data['t'][0], data['t'][-1]])
    
    fig.tight_layout()
    plt.show()

Show results for particular choices of equilibrium point, gain matrix, and initial conditions.

In [ ]:
(q1e, v1e, q2e, v2e, taue) = (np.pi, 0., 0., 0., 0.) # <-- EQUILIBRIUM POINT (q1e = pi*integer, q2e = real, v1e = v2e = taue = 0)
K = np.array([[60., 30., 20., 1.]]) # <-- GAIN MATRIX FOR LINEAR STATE FEEDBACK
(q1i, v1i, q2i, v2i) = (np.pi - 0.05, -0.05, 0., 0.) # <-- INITIAL CONDITIONS

A, B = get_model(q1e, v1e, q2e, v2e, taue)
data = get_data(simulator, q1e, v1e, q2e, v2e, taue, K, q1i, v1i, q2i, v2i, max_time=5.0)
show_results(data, q1e, v1e, q2e, v2e, taue, A, B, K, q1i, v1i, q2i, v2i, show_pred=True)
# # show_x_symbolic(A, B, K)
# print('\nEIGENVALUES OF CLOSED-LOOP SYSTEM\n')
# print(f's = {linalg.eigvals(A - B @ K)}')

In [13]:
### test cell ###

(q1e, v1e, q2e, v2e, taue) = (np.pi, 0., 0., 0., 0.) # <-- EQUILIBRIUM POINT (q1e = pi*integer, q2e = real, v1e = v2e = taue = 0)
# K = np.array([[60., 30., 20., 1.]]) # <-- GAIN MATRIX FOR LINEAR STATE FEEDBACK
(q1i, v1i, q2i, v2i) = (np.pi - 0.05, -0.05, 0., 0.) # <-- INITIAL CONDITIONS

A, B = get_model(q1e, v1e, q2e, v2e, taue)

A, B

(array([[   0.        ,    1.        ,    0.        ,    0.        ],
        [   4.34936821,    0.        ,    0.        ,    2.21680337],
        [   0.        ,    0.        ,    0.        ,    1.        ],
        [   0.        , -909.09090909,    0.        ,    0.        ]]),
 array([[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [90.90909091]]))

In [ ]:
# Find closed-loop system:
F = A - B @ K

# Find matrix S with eigenvalues in the diagonal, and matrix V with corresponding eigenvectors in each column
s, V = linalg.eig(F)
S = np.diag(s)


# Everything above this cell is all that is needed to run the simulation besides a working K. This requires user input.

## Everything below this cell is old code (redundant).

### Delete at your own risk


In [88]:
# Define a function to return a state-space model for a given choice of equilibrium point.
t = sym.symbols('t')
J_1z = sym.symbols('J_1z')
J_2x, J_2z = sym.symbols('J_2x, J_2z')
J_3x, J_3y, J_3z = sym.symbols('J_3x, J_3y, J_3z')
r = sym.symbols('r')
m = sym.symbols('m')
g = sym.symbols('g')
q1, q2 = mechanics.dynamicsymbols('q1, q2')
v1, v2 = sym.symbols('v1, v2') #v = qdot
v1 = q1.diff(t)
v2 = q2.diff(t)
w1, w2 = sym.symbols('w1, w2') #qdd is q double dot
w1 = v1.diff(t)
w2 = v2.diff(t)
a1, a2, a3, a4, a5, a6, a7, a8 = sym.symbols('a1, a2, a3, a4, a5, a6, a7, a8')
tau = sym.symbols('tau') 
J_1z = 0.5
J_2x = 0.001
J_2z = 0.001
J_3x = 0.01
J_3y = 0.01
J_3z = 0.01
m = 1.0
r = 2.0
g = 9.81
a1 = -J_3y + 2 * J_3z
a2 = 2* J_3y 
a3 = -2 * g * m * r
a4 = 2 * J_1z + 2* J_2z + 2 * m * r**2
a5 = 2* J_3z
a6 = (J_3y - J_3z)/(2 * (J_2x + J_3x))
a7 = -J_3y/(J_2x + J_3x)
a8 = 1/(J_2x + J_3x)
v_rotor = 1000.
q1e = 0.
v1e = 0.
q2e = 0.
v2e = 0.
taue = 0.

def get_model(q1e, v1e, q2e, v2e, tau):
    f = sym.Matrix([
        v1, 
        (((a1 * sym.sin(2 * q2) * v1 * v2) + (a2 * sym.cos(q2) * v2*1000) + (a3)*sym.sin(q1)) / (a4 + (a5 * sym.cos(q2)**2))),
        v2,
        a6 * sym.sin(2 * q2) * v1**2 + a7 * sym.cos(q2) * v1*1000 + a8 * tau
    ])
    f = sym.nsimplify(f, rational=True)
    f_num = sym.lambdify([q1, v1, q2, v2, tau], f)
    if not np.allclose(f_num(q1e, v1e, q2e, v2e, taue), 0.):
        raise Exception('equilibrium point is invalid')
    
    A_sym = f.jacobian([q1, v1, q2, v2])
    B_sym = f.jacobian([tau])
    A_num = sym.lambdify([q1, v1, q2, v2, tau], A_sym)
    B_num = sym.lambdify([q1, v1, q2, v2, tau], B_sym)
    A = A_num(q1e, q2e, v1e, v2e, taue).astype(float)
    B = B_num(q1e, q2e, v1e, v2e, taue).astype(float)
    return A, B
A = get_model(q1e, v1e, q2e, v2e, tau)[0]
B = get_model(q1e, v1e, q2e, v2e, tau)[1]

A, B

(array([[   0.        ,    1.        ,    0.        ,    0.        ],
        [  -4.34936821,    0.        ,    0.        ,    2.21680337],
        [   0.        ,    0.        ,    0.        ,    1.        ],
        [   0.        , -909.09090909,    0.        ,    0.        ]]),
 array([[ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [90.90909091]]))

In [14]:
# Find K

i = 0
while i == 0:
    K = np.array([[np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()]])
    F  = A - B@K
    s = linalg.eigvals(F)
    if (s.real < 0).all():
        i += 1
        
print(K.tolist())
print(s)

KeyboardInterrupt: 

Define a controller that maps sensor measurements to actuator commands. By default, it chooses to apply zero torque to the gimbal. Modify the `run` function to make the controller behave differently.

In [1]:
# Find K

i = 0
while i == 0:
    K = np.array([[np.random.rand(),np.random.rand(),np.random.rand(),np.random.rand()]])
    F  = A - B@K
    s = linalg.eigvals(F)
    if (s.real < 0).all() and (s.imag == 0).all():
        i += 1

# count = 0
# while count == 0:
    
#     K = np.zeros((1,4))
#     for i in range(len(K-1)):
#         K[i] = np.random.rand()
#     F  = A - B@K
#     s = linalg.eigvals(F)
#     if (s.real < 0).all() and (s.imag == 0).all():
#         count += 1
print(K.tolist())
print(s)

class Controller:
    def __init__(self, K, q1e, v1e, q2e, v2e, taue):
        self.K = K
        self.q1e = q1e
        self.q2e = q2e
        self.v1e = v1e
        self.v2e = v2e
        self.taue = taue
    
    def reset(self):
        pass
    
    def run(
            self,
            t,
            platform_angle,
            platform_velocity,
            gimbal_angle,
            gimbal_velocity,
            rotor_velocity,
        ):
        
        # Find state
        x = np.array([
            platform_angle - self.q1e,
            platform_velocity - self.v1e,
            gimbal_angle - self.q2e,
            gimbal_velocity - self.v2e,
        ])
        
        # Find input
        u = - self.K @ x
        
        # Find actuator command
        tau = u[0]
        
        gimbal_torque = -tau
        return gimbal_torque
    
def get_data(simulator, q1e, v1e, q2e, v2e, taue, K, q1i, v1i, q2i, v2i, max_time=5.0):
# Create controller
    controller = Controller(K, q1e, v1e, q2e, v2e, taue)

    # Reset simulator with initial conditions [q1i, v1i, q2i, v2i]
    simulator.reset(
        platform_angle = q1i,
        platform_velocity = v1i,
        gimbal_angle = q2i,
        gimbal_velocity = v2i,
        rotor_velocity = v_rotor,
    )

    # Reset controller
    controller.reset()
    
    # Choose a camera view ( simulator.camera_sideview() or simulator.camera_topview() )
    simulator.camera_sideview()

    # Run simulator
    data = simulator.run(
        controller,           # <-- required (an instance of your Controller class)
        max_time=max_time,    # <-- optional (how long you want to run the simulation in seconds)
        data_filename=None,   # <-- optional (name of file to which you want data saved, e.g., 'my_data.json')
        video_filename=None,  # <-- optional (name of file to which you want video saved, e.g., 'my_video.mov')
    )

    # Return data
    return data
    
def get_x_numeric(data, q1e, v1e, q2e, v2e, A, B):
    # Define initial conditions
    x0 = np.array([
        data['platform_angle'][0] - q1e,
        data['platform_velocity'][0] - v1e,
        data['gimbal_angle'][0] - q2e,
        data['gimbal_velocity'][0] - v2e,
    ])

    # Create empty array to hold state at each time
    x = []

    # Compute state at each time
    for t in data['t']:
        x.append(linalg.expm((A - B @ K) * t) @ x0)
    x = np.array(x)
    
    # Return state at each time
    return x

def get_x_symbolic(A, B, K):
    # Model
    A = sym.nsimplify(sym.Matrix(A), rational=True)
    B = sym.nsimplify(sym.Matrix(B), rational=True)
    K = sym.nsimplify(sym.Matrix(K), rational=True)
    
    # Variables
    t, x_1i, x_2i, x_3i, x_4i = sym.symbols('t, x_1i, x_2i, x_3i, x_4i', real=True)
    
    # Initial condition
    x0 = sym.Matrix([x_1i, x_2i, x_3i, x_4i])
    
    # Solution
    x = sym.exp((A - B @ K) * t) @ x0
    return x

def show_x_symbolic(A, B, K):
    x = get_x_symbolic(A, B, K)
    print('\nSOLUTION TO CLOSED-LOOP SYSTEM')
    display(Markdown(f'$$x(t) = {sym.latex(x)}$$'))
    
# def show_results(data, q1e, v1e, q2e, v2e, taue, A, B, K, q1i, v1i, v2i, show_pred=False):
#     t = data['t']
#     q1 = data['platform_angle']
#     v1 = data['platform_velocity']
#     q2 = data['gimbal_angle']
#     v2 = data['gimbal_velocity']
#     x1 = q1 - q1e
#     x2 = v1 - v1e
#     x3 = q2 - q2e
#     x4 = v2 - v2e
    
#     fig, ((ax_q1, ax_x1), (ax_v1, ax_x2), (ax_q2, ax_x3), (ax_v2, ax_x4)) = plt.subplots(4, 2, figsize=(15, 12), sharex=True)

#     ax_q1.plot(t, q1, label='$q_1$', linewidth=4)
#     ax_q1.plot(t, np.ones_like(t) * q1e, '--', label='$q_1$ (desired)', linewidth=4)
    
#     ax_v1.plot(t, v1, label='$v_1$', linewidth=4)
#     ax_v1.plot(t, np.ones_like(t) * v1e, '--', label='$v_1$ (desired)', linewidth=4)
    
#     ax_v2.plot(t, v2, label='$v_2$', linewidth=4)
#     ax_v2.plot(t, np.ones_like(t) * v2e, '--', label='$v_2$ (desired)', linewidth=4)
    
#     ax_x1.plot(t, x1, label='$x_1$', linewidth=4)
#     ax_x1.plot(t, np.zeros_like(t), '--', label='$x_1$ (desired)', linewidth=4)
    
#     ax_x2.plot(t, x2, label='$x_2$', linewidth=4)
#     ax_x2.plot(t, np.zeros_like(t), '--', label='$x_2$ (desired)', linewidth=4)
    
#     ax_x3.plot(t, x3, label='$x_3$', linewidth=4)
#     ax_x3.plot(t, np.zeros_like(t), '--', label='$x_3$ (desired)', linewidth=4)
    
#     if show_pred:
#         # Get x(t) and extract components
#         x_num = get_x_numeric(data, q1e, v1e, v2e, A, B)
#         x1_num = x_num[:, 0]
#         x2_num = x_num[:, 1]
#         x3_num = x_num[:, 2]
        
#         # Compute q1(t), v1(t), and v2(t) from x(t)
#         q1_num = x1_num + q1e
#         v1_num = x2_num + v1e
#         v2_num = x3_num + v2e
        
#         # Add everything to plots
#         ax_q1.plot(t, q1_num, ':', label='$q_1$ (linear)', linewidth=6, color='C3')
#         ax_v1.plot(t, v1_num, ':', label='$v_1$ (linear)', linewidth=6, color='C3')
#         ax_v2.plot(t, v2_num, ':', label='$v_2$ (linear)', linewidth=6, color='C3')
#         ax_x1.plot(t, x1_num, ':', label='$x_1$ (linear)', linewidth=6, color='C3')
#         ax_x2.plot(t, x2_num, ':', label='$x_2$ (linear)', linewidth=6, color='C3')
#         ax_x3.plot(t, x3_num, ':', label='$x_3$ (linear)', linewidth=6, color='C3')
        
#     ax_q1.grid()
#     ax_q1.legend(fontsize=16)
#     ax_q1.tick_params(labelsize=14)
#     ax_q1.set_ylim(np.pi - 0.5, np.pi + 0.5)
    
#     ax_v1.grid()
#     ax_v1.legend(fontsize=16)
#     ax_v1.tick_params(labelsize=14)
#     ax_v1.set_ylim(-1, 1)
    
#     ax_v2.grid()
#     ax_v2.legend(fontsize=16)
#     ax_v2.tick_params(labelsize=14)
#     ax_v2.set_ylim(-5, 5)

#     ax_x1.grid()
#     ax_x1.legend(fontsize=16)
#     ax_x1.tick_params(labelsize=14)
#     ax_x1.set_ylim(-0.5, 0.5)
        
#     ax_x2.grid()
#     ax_x2.legend(fontsize=16)
#     ax_x2.tick_params(labelsize=14)
#     ax_x2.set_ylim(-1, 1)
    
#     ax_x3.grid()
#     ax_x3.legend(fontsize=16)
#     ax_x3.tick_params(labelsize=14)
#     ax_x3.set_ylim(-5, 5)
    
#     ax_v2.set_xlabel('time (s)', fontsize=20)
#     ax_v2.set_xlim([data['t'][0], data['t'][-1]])
#     ax_x3.set_xlabel('time (s)', fontsize=20)
#     ax_x3.set_xlim([data['t'][0], data['t'][-1]])
    
#     fig.tight_layout()
#     plt.show()

NameError: name 'np' is not defined

In [92]:
(q1i, v1i, q2i, v2i) = (np.pi/5, 0, 0,0)

A, B = get_model(q1e, v1e, q2e, v2e, tau)
data = get_data(simulator, q1e, v1e, q2e, v2e, taue, K, q1i, v1i, q2i, v2i, max_time=10.0)

In [ ]:
# [[0.9952619413213912, 0.05536381735881257, 0.06186576991565962, 0.9952144494685199]]